# Instalo las librerias necesarias

In [1]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.8 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [3]:
from sklearn.utils import resample

In [4]:
from category_encoders import CatBoostEncoder

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
from sklearn.model_selection import train_test_split

# Se cargan los archivos de entrenamiento analizados anteriormente

In [7]:
parquet_file = '/content/drive/MyDrive/Colab Notebooks/TP2/RandomForest (Modelo2)/train_set_modelos_analizado.parq'

In [8]:
data_set = pd.read_parquet(parquet_file)

In [9]:
data_set

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,mean_per_ipenum,count_per_ipenum,attackerip_per_country,attackerip_per_name,attackerip_per_time,label
0,7.283333,DE,Host Europe GmbH,TR,Murat Aktas,http:exploit,0,6466,6470.080585,121065,60279,6470,90401,0
1,7.283333,DE,Host Europe GmbH,TR,Murat Aktas,http:spam,0,6466,6470.080585,121065,60279,6470,90401,0
2,7.283333,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:bruteforce,2,4637,84007.011771,537782,95007,83712,85982,0
3,7.283333,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:spam,2,4637,84007.011771,537782,95007,83712,85982,0
4,7.283333,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:exploit,2,4637,84007.011771,537782,95007,83712,85982,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40041183,10.683333,US,LIQUIDWEB,US,UNIFIEDLAYER-AS-1,http:scan,14494,196833,95940.124103,255666,97204,96108,132991,0
40041184,10.683333,DE,Host Europe GmbH,SG,AMAZON-02,http:scan,5578,190687,95698.728470,564852,107871,95121,128542,0
40041185,10.683333,DE,Host Europe GmbH,SG,AMAZON-02,http:spam,5578,190687,95698.728470,564852,107871,95121,128542,0
40041186,10.683333,AU,Aussie Broadband,AU,Host Universal Pty Ltd,tcp:scan,17620,193153,179502.250614,10171,103948,163453,102139,0


#Realizo el split

La forma de hacer el split se hizo basandonos en la informacion que obtuvimos del BASELINE en donde notamos la importantacia que aparentemente se le estaba dando a attacker_as_name

In [10]:
def split(data_set):

  #Separo las ips para que no se repitan entre train y test
  unique_ips = data_set['attacker_ip_enum'].unique()

  #Obtengo las claves de los attacker_as_name que tengan mas cero
  top_attack_types_0_base = data_set[data_set['label'] == 0]['attacker_as_name'].value_counts()
  top_attack_types_0_top50 = top_attack_types_0_base.head(50)
  top_attack_types_0_top20 = top_attack_types_0_base.head(20)

  #Obtengo las claves de los attacker_as_name que tengan mas uno PERO que no pertenezcan a los que tienen mas cero
  top_attack_types_1_base = data_set[data_set['label'] == 1]['attacker_as_name'].value_counts().head(50)
  top_attack_types_1_base = top_attack_types_1_base[~top_attack_types_1_base.index.isin(top_attack_types_0_top50.index)]
  top_attack_types_1_top40 =top_attack_types_1_base.head(40)

  #Uno las claves obtenidas anteriormente y me quedo con esos fragmentos de mi df ya que se consideran los mas representativos de cada label
  merged_indices = top_attack_types_0_top20.index.union(top_attack_types_1_top40.index)
  data_set = data_set[data_set['attacker_as_name'].isin(merged_indices)]

  #Hago mi split en base a las ips para que no se repitan entre train y test
  train_ips, test_ips = train_test_split(unique_ips, test_size=0.20, random_state=7)

  # Filtro el DataFrame original para obtener conjuntos de entrenamiento y prueba
  train_set = data_set[data_set['attacker_ip_enum'].isin(train_ips)]
  test_set = data_set[data_set['attacker_ip_enum'].isin(test_ips)]

  #Elimino las ip de los atacantes de mis set de entrenamiento y prueba
  train_set = train_set.drop(columns=['attacker_ip_enum'])
  test_set = test_set.drop(columns=['attacker_ip_enum'])

  #Hago un undersample
  clase_mayoritaria = train_set[train_set['label'] == 0]
  clase_minoritaria = train_set[train_set['label'] == 1]
  clase_mayoritaria_undersampled = resample(clase_mayoritaria, replace=False, n_samples=(len(clase_minoritaria))*6, random_state=50)
  train_set = pd.concat([clase_mayoritaria_undersampled, clase_minoritaria])

  #Reseteo el index de ambos set de datos
  train_set = train_set.reset_index(drop=True)
  test_set = test_set.reset_index(drop=True)

  #Separo en X e Y
  X_train, y_train = train_set.iloc[:,:-1],train_set.iloc[:,-1]
  X_test, y_test = test_set.iloc[:,:-1],test_set.iloc[:,-1]

  return X_train, y_train, X_test, y_test

In [11]:
X_train, y_train, X_test, y_test =split(data_set)

In [12]:
del data_set

In [13]:
X_train

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,mean_per_ipenum,count_per_ipenum,attackerip_per_country,attackerip_per_name,attackerip_per_time
0,19.483334,CZ,Datacamp Limited,US,"Tencent Building, Kejizhongyi Avenue",ssh:bruteforce,297,114572.026708,107310,97204,107536,57642
1,10.066667,DE,Host Europe GmbH,NO,M247 Europe SRL,http:exploit,10968,116212.737455,57274,91073,91691,82012
2,20.116667,DE,Hetzner Online GmbH,DE,AMAZON-02,http:scan,13159,53974.533221,96943,95007,95121,49803
3,18.883333,US,A2HOSTING,FR,OVH SAS,http:spam,2339,106068.092660,1505863,103420,109682,106260
4,9.600000,SG,AS-26496-GO-DADDY-COM-LLC,NL,GOOGLE-CLOUD-PLATFORM,http:scan,132,109471.905231,458337,111663,113613,96644
...,...,...,...,...,...,...,...,...,...,...,...,...
3488648,10.600000,US,GOOGLE-CLOUD-PLATFORM,DE,F3 Netze e.V.,http:scan,15892,153802.808311,9505,95007,153802,88080
3488649,10.616667,US,PRIVATESYSTEMS,SG,DIGITALOCEAN-ASN,http:spam,65013,106116.509835,578169,107871,94579,102824
3488650,10.616667,US,PRIVATESYSTEMS,SG,DIGITALOCEAN-ASN,http:scan,65013,106116.509835,578169,107871,94579,102824
3488651,10.666667,FR,OVH SAS,SG,DIGITALOCEAN-ASN,http:spam,2244,106116.509835,578169,107871,94579,82314


#Encode

#Encode del name

In [14]:
encoderCatBoost_asname = CatBoostEncoder(cols=['attacker_as_name','watcher_as_name'])

encoderCatBoost_asname.fit(X_train, y_train)
X_train = encoderCatBoost_asname.transform(X_train)
X_test = encoderCatBoost_asname.transform(X_test)

#Encode de attack_type

In [15]:
ohencoder = OneHotEncoder(drop='first', dtype='int8')
encoded_emb_train = ohencoder.fit_transform(X_train[['attack_type']]).todense().astype(int)
encoded_emb_test = ohencoder.transform(X_test[['attack_type']]).todense().astype(int)

new_column_names = ohencoder.get_feature_names_out(['attack_type'])

X_train = X_train.reset_index().drop(['index', 'attack_type'],axis=1).join(pd.DataFrame(encoded_emb_train))
X_test = X_test.reset_index().drop(['index', 'attack_type'],axis=1).join(pd.DataFrame(encoded_emb_test))

In [16]:
for i in range(14):
    X_train.rename(columns={X_train.columns[i+11]: new_column_names[i]}, inplace=True)
    X_test.rename(columns={X_test.columns[i+11]: new_column_names[i]}, inplace=True)

In [17]:
del encoded_emb_train
del encoded_emb_test

#Encode de country

In [18]:
encoderCatBoost_country = CatBoostEncoder(cols=['watcher_country','attacker_country'])

encoderCatBoost_country.fit(X_train, y_train)
X_train = encoderCatBoost_country.transform(X_train)
X_test = encoderCatBoost_country.transform(X_test)

#Guardo el set de train

In [19]:
set_train = pd.concat([X_train, y_train], axis=1)
set_train.to_parquet('/content/drive/MyDrive/Colab Notebooks/TP2/RandomForest (Modelo2)/set_train_modelos_encoded.parq')

del X_train
del y_train
del set_train

#Guardo el set de validacion

In [20]:
set_validation = pd.concat([X_test, y_test], axis=1)
set_validation.to_parquet('/content/drive/MyDrive/Colab Notebooks/TP2/RandomForest (Modelo2)/set_validation_modelos_encoded.parq')

del X_test
del y_test
del set_validation

#Kaggle_set

Cargamos el archivo de kaggle analizado

In [21]:
parquet_file = '/content/drive/MyDrive/Colab Notebooks/TP2/RandomForest (Modelo2)/test_set_modelos_analizado.parq'

test_set = pd.read_parquet(parquet_file)

test_set

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,mean_per_ipenum,count_per_ipenum,attackerip_per_country,attackerip_per_name,attackerip_per_time
0,7.283333,US,IMH-IAD,IN,DIGITALOCEAN-ASN,http:spam,1,7696,79706.133121,344717,84583,102034,43101
1,7.283333,US,IMH-IAD,IN,DIGITALOCEAN-ASN,http:exploit,1,7696,79706.133121,344717,84583,102034,43101
2,7.283333,US,IMH-IAD,IN,DIGITALOCEAN-ASN,http:bruteforce,1,7696,79706.133121,344717,84583,102034,43101
3,7.283333,US,IMH-IAD,IN,DIGITALOCEAN-ASN,http:bruteforce,1,7696,79706.133121,344717,84583,102034,43101
4,7.283333,US,GOOGLE-CLOUD-PLATFORM,US,PONYNET,http:bruteforce,3,7543,112075.389346,27708,100408,121225,17808
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18682292,21.233334,DE,ENTEGA Medianet GmbH,DE,Deutsche Telekom AG,http:scan,24879,198081,129279.870083,267471,106526,129287,105275
18682293,21.233334,DE,ENTEGA Medianet GmbH,DE,Deutsche Telekom AG,http:scan,24879,198081,129279.870083,267471,106526,129287,153738
18682294,21.233334,DE,ENTEGA Medianet GmbH,DE,Deutsche Telekom AG,http:scan,24879,198081,129279.870083,267471,106526,129287,105275
18682295,21.233334,DE,ENTEGA Medianet GmbH,DE,Deutsche Telekom AG,http:scan,24879,198081,129279.870083,267471,106526,129287,105275


In [22]:
attacker_id = test_set['attacker_ip_enum']
test_set = test_set.drop('attacker_ip_enum', axis=1)

#Encode

#Encode del name KAGGLE

In [23]:
test_set = encoderCatBoost_asname.transform(test_set)

In [24]:
del encoderCatBoost_asname

#Encode de attack_type KAGGLE


In [25]:
encoded_emb_test = ohencoder.transform(test_set[['attack_type']]).todense().astype(int)
new_column_names = ohencoder.get_feature_names_out(['attack_type'])
test_set = test_set.reset_index().drop(['index', 'attack_type'],axis=1).join(pd.DataFrame(encoded_emb_test))

for i in range(14):
    test_set.rename(columns={test_set.columns[i+11]: new_column_names[i]}, inplace=True)

In [26]:
del encoded_emb_test

In [27]:
del ohencoder

#Encode de country KAGGLE

In [28]:
test_set = encoderCatBoost_country.transform(test_set)

In [29]:
del encoderCatBoost_country

#Almacenamos

In [30]:
test_set = pd.concat([test_set, attacker_id], axis=1)

test_set.to_parquet('/content/drive/MyDrive/Colab Notebooks/TP2/RandomForest (Modelo2)/test_kaggle_modelos_encoded.parq')